In [1]:
import pandas as pd
import pygplates
import numpy as np


csfile = '/Applications/GPlates-2.0.0/SampleData/FeatureCollections/Coastlines/Matthews_etal_GPC_2016_Coastlines_Polyline.gpmlz'
cs = pygplates.FeatureCollection(csfile)


In [2]:
def from_gpml(filename):
    # function to read in any gplates-compatible feature collection and 
    # place it into a pandas dataframe

    feature_collection = pygplates.FeatureCollection(filename)

    #gpdf = GPlatesDataFrame()

    #DataFrameTemplate = ['lon','lat','geometry']
    DataFrameTemplate = ['feature','geometry','Longitude','Latitude','Name','Description','FromAge','ToAge','PlateID','Type']


    # Get attribute (other than coordinate) names from first feature
    for feature in feature_collection:
        for attribute in feature.get_shapefile_attributes():
            DataFrameTemplate.append(attribute) 
        break

    res = []
    for feature in feature_collection:
        for geometry in feature.get_geometries():
            tmp = []
            tmp.append(feature)
            tmp.append(geometry)
            tmp.append(geometry.get_centroid().to_lat_lon_point().get_longitude())
            tmp.append(geometry.get_centroid().to_lat_lon_point().get_latitude())
            tmp.append(feature.get_name())
            tmp.append(feature.get_description())
            tmp.append(feature.get_valid_time()[0])
            tmp.append(feature.get_valid_time()[1])
            tmp.append(feature.get_reconstruction_plate_id())
            tmp.append(feature.get_feature_type())
            for attribute in feature.get_shapefile_attributes():
                tmp.append(feature.get_shapefile_attribute(attribute))
            res.append(tmp)

    df = pd.DataFrame(res,columns=DataFrameTemplate)

    return df


In [3]:
class GPlatesDataFrame(pd.DataFrame):

    # Pandas inheritance implementation details...
    _metadata = ['filename']
    @property
    def _constructor(self):
        return GPlatesDataFrame
    
    
    def __init__(self, *args, **kwargs):
        # if filename is specified, store it. If not, set filename to None
        filename = kwargs.pop('gplates_filename', None)
        
        # Construct pandas dataframe base/super object
        super(GPlatesDataFrame, self).__init__(*args, **kwargs)
        
        # add the filename to the derived object
        self.filename = filename
      
    @classmethod
    def from_gpml(cls, filename):
        # function to read in any gplates-compatible feature collection and 
        # place it into a pandas dataframe

        feature_collection = pygplates.FeatureCollection(filename)
        
        #DataFrameTemplate = ['lon','lat','geometry']
        DataFrameTemplate = ['feature','geometry','Longitude','Latitude','Name','Description','FromAge','ToAge','PlateID','Type']


        # Get attribute (other than coordinate) names from first feature
        for feature in feature_collection:
            for attribute in feature.get_shapefile_attributes():
                DataFrameTemplate.append(attribute) 
            break

        res = []
        for feature in feature_collection:
            for geometry in feature.get_geometries():
                tmp = []
                tmp.append(feature)
                tmp.append(geometry)
                tmp.append(geometry.get_centroid().to_lat_lon_point().get_longitude())
                tmp.append(geometry.get_centroid().to_lat_lon_point().get_latitude())
                tmp.append(feature.get_name())
                tmp.append(feature.get_description())
                tmp.append(feature.get_valid_time()[0])
                tmp.append(feature.get_valid_time()[1])
                tmp.append(feature.get_reconstruction_plate_id())
                tmp.append(feature.get_feature_type())
                for attribute in feature.get_shapefile_attributes():
                    tmp.append(feature.get_shapefile_attribute(attribute))
                res.append(tmp)

        return GPlatesDataFrame(res, columns=DataFrameTemplate, gplates_filename=filename)
    
    def to_file(self, filename):
        print 'do some stuff to write a file'



csfile = '/Applications/GPlates-2.0.0/SampleData/FeatureCollections/Coastlines/Matthews_etal_GPC_2016_Coastlines_Polyline.gpmlz'
cs = pygplates.FeatureCollection(csfile)

test = GPlatesDataFrame.from_gpml(csfile)

type(test)

test.to_file('blah')


do some stuff to write a file


In [4]:
test2 = test[test.Name.str.contains('Africa')]

print type(test2)

test2.filename


<class '__main__.GPlatesDataFrame'>


'/Applications/GPlates-2.0.0/SampleData/FeatureCollections/Coastlines/Matthews_etal_GPC_2016_Coastlines_Polyline.gpmlz'